In [5]:
import numpy as np
from scipy.spatial.distance import euclidean
import cv2
import dlib
import matplotlib.pyplot as plt

# Function to calculate DTW distance
def calculate_dtw_distance(sequence1, sequence2):
    # Normalize the sequences
    sequence1 = sequence1 / np.linalg.norm(sequence1, axis=1, keepdims=True)
    sequence2 = sequence2 / np.linalg.norm(sequence2, axis=1, keepdims=True)

    # Initialize the DTW matrix
    dtw_matrix = np.zeros((len(sequence1), len(sequence2)))

    # Fill the DTW matrix
    for i in range(len(sequence1)):
        for j in range(len(sequence2)):
            cost = euclidean(sequence1[i], sequence2[j])
            if i == 0 and j == 0:
                dtw_matrix[i][j] = cost
            elif i == 0:
                dtw_matrix[i][j] = cost + dtw_matrix[i][j - 1]
            elif j == 0:
                dtw_matrix[i][j] = cost + dtw_matrix[i - 1][j]
            else:
                dtw_matrix[i][j] = cost + min(
                    dtw_matrix[i - 1][j],        # insertion
                    dtw_matrix[i][j - 1],        # deletion
                    dtw_matrix[i - 1][j - 1]     # match
                )

    return dtw_matrix[-1][-1]

# Function to detect facial landmarks
def detect_landmarks(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Load the pre-trained facial landmark model
    predictor = dlib.shape_predictor("/content/shape_predictor_81_face_landmarks.dat")
    detector = dlib.get_frontal_face_detector()

    # Detect faces in the image
    faces = detector(gray)

    # Assume only one face is present in the image
    if len(faces) == 0:
        print("No faces detected.")
        return None

    # Get the facial landmarks for the first face
    shape = predictor(gray, faces[0])
    landmarks = []
    for i in range(68):
        x, y = shape.part(i).x, shape.part(i).y
        landmarks.append([x, y])

    return np.array(landmarks)

# Function to plot the image with landmarks
def plot_image_with_landmarks(image, landmarks):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.scatter(landmarks[:, 0], landmarks[:, 1], color='red', s=5)
    plt.axis('off')
    plt.show()

# Load the reference and unknown face images
reference_image_path = "/content/AJ_1 (1).jpg"
unknown_image_path = "/content/AJ_1 (1).jpg"

#reference_image_path = "/content/BP_1.jpg"
#unknown_image_path = "/content/BP_2.jpg"

#reference_image_path = "/content/JDP_1 (1).jpg"
#unknown_image_path = "/content/JDP_1 (1).jpg"

#reference_image_path = "/content/dr_1.JPG"
#unknown_image_path = "/content/dr_2.jpg"

# Detect landmarks for the reference and unknown images
reference_landmarks = detect_landmarks(reference_image_path)
unknown_landmarks = detect_landmarks(unknown_image_path)

# Check if landmarks were detected
if reference_landmarks is None or unknown_landmarks is None:
    exit()

# Load the images for plotting
reference_image = cv2.imread(reference_image_path)
unknown_image = cv2.imread(unknown_image_path)

# Plot the images with landmarks
plot_image_with_landmarks(reference_image, reference_landmarks)
plot_image_with_landmarks(unknown_image, unknown_landmarks)

# Calculate DTW distance between the landmarks
dtw_distance = calculate_dtw_distance(reference_landmarks, unknown_landmarks)

# Set a threshold for face recognition (you may need to adjust this based on your data)
threshold = 10  # Adjust this threshold accordingly
#threshold = 5

# Perform face recognition
if dtw_distance < threshold:
    print("Face recognized! DTW Distance:", dtw_distance)
else:
    print("Face not recognized. DTW Distance:", dtw_distance)

ModuleNotFoundError: No module named 'dlib'